# Tower Input Data for PM-JPL from ECOSTRESS Collection 2 Cal-Val

This notebook demonstrates how to generate input data for the Penman-Monteith Jet Propulsion Laboratory (PM-JPL) evapotranspiration model using ECOSTRESS Collection 2 calibration and validation tower data.

## Overview

The PM-JPL model is an implementation of the Penman-Monteith equation for estimating evapotranspiration. This notebook:

1. Loads calibration/validation tower data from ECOSTRESS Collection 2
2. Loads static input parameters for PM-JPL
3. Generates additional required input variables (Topt, fAPARmax, canopy height, soil properties)
4. Processes the data through the PM-JPL model
5. Saves the input and output tables for analysis

## Requirements

- PM-JPL package
- ECOSTRESS calibration/validation tables
- Additional geospatial data sources for model inputs

In [1]:
import os
from rasters import MultiPoint
from PMJPL import process_PMJPL_table
from PMJPL import load_ECOv002_static_tower_PMJPL_inputs
from ECOv002_calval_tables import load_calval_table

In [2]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'PMJPL')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PM-JPL-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PM-JPL-outputs.csv")

In [3]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C,SWin_Wm2,emissivity
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,545.51056,0.948
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,848.34390,0.952
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,838.81160,0.972
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,851.72480,0.974
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,702.55160,0.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,286.84660,0.980
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,290.87400,0.976
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,352.19530,0.976
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,420.67880,0.976


Filter out freezing temperatures

In [4]:
model_inputs_gdf = model_inputs_gdf[(model_inputs_gdf['Ta_C'] > 0) & (model_inputs_gdf['ST_C'] > 0)]
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C,SWin_Wm2,emissivity
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,545.51056,0.948
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,848.34390,0.952
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,838.81160,0.972
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,851.72480,0.974
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,702.55160,0.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,286.84660,0.980
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,290.87400,0.976
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,352.19530,0.976
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,420.67880,0.976


In [5]:
Tmin_C_by_tower = model_inputs_gdf.groupby('ID')['Ta_C'].min().reset_index().rename(columns={'Ta_C': 'Tmin_C'})
Tmin_C_by_tower

,ID,Tmin_C
0,CA-Cbo,15.979751
1,PR-xGU,27.538052
2,US-ARM,9.637029
3,US-CMW,9.046284
4,US-CS5,15.153473
...,...,...
58,US-xTR,14.414353
59,US-xUK,2.654168
60,US-xUN,11.620493
61,US-xWR,9.558230


In [6]:
static_inputs_df = load_ECOv002_static_tower_PMJPL_inputs()
static_inputs_df

,ID,name,elevation_km,gl_sh,gl_e_wv,RBL_min,RBL_max,CL,Tmin_open,Tmin_closed,VPD_open,VPD_closed,geometry
0,US-NC3,NC_Clearcut#3,0.005,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-76.656 35.799)
1,PE-QFR,Quistococha Forest Reserve,0.104,0.01,0.01,70.0,100.0,0.0025,9.09,-8.0,1000.0,4000.0,POINT (-73.319 -3.8344)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,0.270,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,POINT (-80.637 41.8222)
3,US-NC4,NC_AlligatorRiver,0.001,0.04,0.04,65.0,95.0,0.0025,9.50,-7.0,650.0,2900.0,POINT (-75.9038 35.7879)
4,CA-DB2,Delta Burns Bog 2,0.004,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,POINT (-122.9951 49.119)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",1.364,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,POINT (-103.0293 40.4619)
117,US-xWD,NEON Woodworth (WOOD),0.579,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-99.2414 47.1282)
118,US-CS4,Central Sands Irrigated Agricultural Field,0.328,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-89.5475 44.1597)
119,US-xAE,NEON Klemme Range Research Station (OAES),0.516,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,POINT (-99.0588 35.4106)


In [7]:
static_inputs_df = static_inputs_df.merge(Tmin_C_by_tower, on="ID", how="left")
static_inputs_df

,ID,name,elevation_km,gl_sh,gl_e_wv,RBL_min,RBL_max,CL,Tmin_open,Tmin_closed,VPD_open,VPD_closed,geometry,Tmin_C
0,US-NC3,NC_Clearcut#3,0.005,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-76.656 35.799),32.658920
1,PE-QFR,Quistococha Forest Reserve,0.104,0.01,0.01,70.0,100.0,0.0025,9.09,-8.0,1000.0,4000.0,POINT (-73.319 -3.8344),NaN
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,0.270,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,POINT (-80.637 41.8222),20.566360
3,US-NC4,NC_AlligatorRiver,0.001,0.04,0.04,65.0,95.0,0.0025,9.50,-7.0,650.0,2900.0,POINT (-75.9038 35.7879),20.168324
4,CA-DB2,Delta Burns Bog 2,0.004,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,POINT (-122.9951 49.119),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",1.364,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,POINT (-103.0293 40.4619),11.471415
117,US-xWD,NEON Woodworth (WOOD),0.579,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-99.2414 47.1282),NaN
118,US-CS4,Central Sands Irrigated Agricultural Field,0.328,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,POINT (-89.5475 44.1597),NaN
119,US-xAE,NEON Klemme Range Research Station (OAES),0.516,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,POINT (-99.0588 35.4106),3.815752


In [8]:
# extract array of x and array of y coordinates from data frame geometry
x_coords = model_inputs_gdf.geometry.x
y_coords = model_inputs_gdf.geometry.y
tower_points = MultiPoint(x=x_coords, y=y_coords)
tower_points

MULTIPOINT ((-76.656 35.799), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-75.9038 35.7879), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-90.3004 45.9793), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6

In [9]:
# merge static inputs with model inputs, ignoring duplicate columns from static_inputs_df
cols_to_use = [col for col in static_inputs_df.columns if col not in model_inputs_gdf.columns or col == 'ID']
model_inputs_gdf = model_inputs_gdf.merge(static_inputs_df[cols_to_use], on="ID", how="left")
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,gl_sh,gl_e_wv,RBL_min,RBL_max,CL,Tmin_open,Tmin_closed,VPD_open,VPD_closed,Tmin_C
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4500.0,32.658920
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,20.566360
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,20.566360
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,20.566360
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,0.08,0.08,25.0,45.0,0.0065,11.39,-8.0,650.0,3500.0,20.566360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,3.815752
1043,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,3.815752
1044,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,3.815752
1045,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,0.02,0.02,20.0,50.0,0.0070,12.02,-8.0,650.0,4200.0,3.815752


In [10]:
model_inputs_gdf.keys()

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C', 'SWin_Wm2', 'emissivity', 'name', 'elevation_km', 'gl_sh',
       'gl_e_wv',

In [11]:
model_inputs_gdf.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1042    0.092853
1043    0.111844
1044    0.106782
1045    0.106775
1046    0.113165
Name: albedo, Length: 1047, dtype: float64

In [12]:
results = process_PMJPL_table(model_inputs_gdf)
results

[2025-09-08 16:21:16 INFO] starting PM-JPL table processing
[2025-09-08 16:21:16 INFO] started extracting geometry from PM-JPL input table
[2025-09-08 16:21:16 INFO] completed extracting geometry from PM-JPL input table
[2025-09-08 16:21:16 INFO] started extracting time from PM-JPL input table
[2025-09-08 16:21:16 INFO] completed extracting time from PM-JPL input table
[2025-09-08 16:21:25 INFO] using given shortwave radiation (SWin_Wm2)
[2025-09-08 16:21:25 INFO] running instantaneous Verma net radiation
[2025-09-08 16:21:25 INFO] starting Verma net radiation processing
[2025-09-08 16:21:25 INFO] variable SWout_Wm2 min: 0.000 mean: 66.238 max: 278.058 nan: 0.00% (nan)
[2025-09-08 16:21:25 INFO] variable SWnet_Wm2 min: 0.000 mean: 545.317 max: 929.140 nan: 0.00% (nan)
[2025-09-08 16:21:25 INFO] variable LWin_Wm2 min: 227.974 mean: 345.942 max: 473.521 nan: 0.00% (nan)
[2025-09-08 16:21:25 INFO] variable LWout_Wm2 min: 308.756 mean: 465.306 max: 693.334 nan: 0.00% (nan)
[2025-09-08 16:2

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,rtotc,rtot,rrs,ras,Asoil,wet_soil_evaporation_Wm2,potential_soil_evaporation_Wm2,fSM,LEs,LE_Wm2
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,38.200872,35.510642,179.536728,29.646791,0.000000,0.000000,0.000000,1.891990e-03,0.000000,299.559254
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,38.090796,38.555330,207.744934,32.519959,0.000000,0.000000,0.000000,1.706940e-03,0.000000,263.840936
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,40.562869,40.602210,202.648575,33.825091,0.000000,0.000000,0.000000,4.747452e-02,0.000000,252.630057
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,42.371784,43.310392,212.608566,35.980767,0.000000,0.000000,0.000000,8.928188e-02,0.000000,280.491994
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,40.988642,41.714993,210.457016,34.814383,0.000000,0.000000,0.000000,3.283667e-02,0.000000,263.373607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,50.000000,58.951465,283.579712,48.805600,80.954786,0.009858,98.565845,6.076479e-02,5.999191,20.683094
1043,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,44.135573,47.507790,229.279434,39.353547,86.726430,0.017991,179.889198,2.028228e-03,0.382847,16.416791
1044,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,28.929644,28.989061,193.958206,25.219714,92.969470,0.029575,295.716840,4.581615e-08,0.029588,28.225251
1045,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,48.531004,54.982557,258.345825,45.334272,101.426627,0.012458,124.563016,1.309988e-02,1.644219,26.537851


In [13]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [14]:
results.to_csv(generated_output_table_filename, index=False)